<a href="https://colab.research.google.com/github/nguyenkien1402/llamaindex-practices/blob/main/pdf-json-extraction/adbove_pdf_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
from google.colab import drive
MOUNTPOINT = '/content/gdrive'
DATADIR = os.path.join(MOUNTPOINT, 'MyDrive', 'llm-poc')
drive.mount(MOUNTPOINT)

Mounted at /content/gdrive


In [1]:
!pip install -r gdrive/MyDrive/llm-doc/requirements.txt

  Using cached camelot_py-0.11.0-py3-none-any.whl (40 kB)
  Obtaining dependency information for llama-index from https://files.pythonhosted.org/packages/c8/29/5507449a06e4af19aa1965ee8ef9368d163feaa1f1ffab38e10017ddd073/llama_index-0.8.36-py3-none-any.whl.metadata
  Using cached llama_index-0.8.36-py3-none-any.whl.metadata (5.0 kB)
  Obtaining dependency information for langchain==0.0.292 from https://files.pythonhosted.org/packages/86/dc/e2c7599b03455ea675107472b60e836f693ac8453e7c1119b61e6e60f574/langchain-0.0.292-py3-none-any.whl.metadata
  Using cached langchain-0.0.292-py3-none-any.whl.metadata (14 kB)
  Obtaining dependency information for transformers==4.33.2 from https://files.pythonhosted.org/packages/1a/06/3817f9bb923437ead9a794f0ac0d03b8b5e0478ab112db4c413dd37c09da/transformers-4.33.2-py3-none-any.whl.metadata
  Using cached transformers-4.33.2-py3-none-any.whl.metadata (119 kB)
  Obtaining dependency information for openai==0.28.0 from https://files.pythonhosted.org/packag

# Use Normal Vector Index

In [59]:
import pandas as pd
from llama_index.evaluation import DatasetGenerator, RelevancyEvaluator, ResponseEvaluator, FaithfulnessEvaluator, QueryResponseEvaluator
from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ServiceContext,
    LLMPredictor,
    Response,
    StorageContext,
    load_index_from_storage,
    SummaryIndex
)

from llama_index.node_parser import SimpleNodeParser
from llama_index.storage.index_store import SimpleIndexStore
from llama_index.prompts import Prompt
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.schema import IndexNode
from llama_index.agent import OpenAIAgent

# define recursive retriever
from llama_index.retrievers import RecursiveRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.response_synthesizers import get_response_synthesizer


import chromadb
from llama_index.vector_stores import ChromaVectorStore

import openai
from llama_index.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from llama_index.embeddings import OpenAIEmbedding

import time
import asyncio
import nest_asyncio
nest_asyncio.apply()

openai.api_key = ""
# os.environ["OPENAI_API_KEY"] = ""
# openai.api_key = os.environ[""]

In [6]:
from llama_index import download_loader
from llama_index import SimpleDirectoryReader

PyMuPDFReader = download_loader('PyMuPDFReader')
loader = PyMuPDFReader()

In [8]:
file_path = "./data/The World's Billionaires - Wikipedia.pdf"
documents = loader.load(file_path="./data/The World's Billionaires - Wikipedia.pdf", metadata=True)

In [60]:
embed_model = OpenAIEmbedding(embed_batch_size=10)
llm_predictor = LLMPredictor(
    llm=OpenAI(temperature=0, model_name="gpt-4", streaming=True)
)
service_context = ServiceContext.from_defaults(
    llm_predictor=llm_predictor,
    embed_model=embed_model
)
doc_nodes = service_context.node_parser.get_nodes_from_documents(documents)

In [61]:
vector_index = VectorStoreIndex(doc_nodes)

In [62]:
query_engine = vector_index.as_query_engine()

In [63]:
response = query_engine.query("How many billionaires were there in 2022?")
response.response

'Based on the context information, the number of billionaires in 2022 is not provided.'

# Use Adobe API

In [35]:
from adobe.pdfservices.operation.auth.credentials import Credentials
from adobe.pdfservices.operation.exception.exceptions import ServiceApiException, ServiceUsageException, SdkException
from adobe.pdfservices.operation.execution_context import ExecutionContext
from adobe.pdfservices.operation.io.file_ref import FileRef
from adobe.pdfservices.operation.pdfops.extract_pdf_operation import ExtractPDFOperation
from adobe.pdfservices.operation.pdfops.options.extractpdf.extract_pdf_options import ExtractPDFOptions
from adobe.pdfservices.operation.pdfops.options.extractpdf.extract_element_type import ExtractElementType

import os.path
import zipfile
import json

In [36]:
#Initial setup, create credentials instance.
client_id = ''
client_secret = ''

credentials = Credentials.service_principal_credentials_builder()\
    .with_client_id(client_id)\
    .with_client_secret(client_secret)\
    .build();

#Create an ExecutionContext using credentials and create a new operation instance.
execution_context = ExecutionContext.create(credentials)

In [37]:
extract_pdf_operation = ExtractPDFOperation.create_new()

#Set operation input from a source file.
source = FileRef.create_from_local_file(file_path)
extract_pdf_operation.set_input(source)

#Build ExtractPDF options and set them into the operation
extract_pdf_options: ExtractPDFOptions = ExtractPDFOptions.builder() \
    .with_element_to_extract(ExtractElementType.TEXT) \
    .build()
extract_pdf_operation.set_options(extract_pdf_options)

In [39]:
#Execute the operation.

zip_file = "./result/ExtractTextInfoFromPDF.zip"
result: FileRef = extract_pdf_operation.execute(execution_context)

#Save the result to the specified location.
result.save_as(zip_file)

OSError: [WinError 17] The system cannot move the file to a different disk drive: 'C:\\Users\\nguye\\AppData\\Local\\Temp\\sdk_result\\01e2a3355e6211eeb636ccd9ac1ec46c.zip' -> 'd:\\works\\llamaindex-practices\\pdf-json-extraction\\result\\ExtractTextInfoFromPDF.zip'

In [40]:
archive = zipfile.ZipFile(zip_file, 'r')
jsonentry = archive.open('structuredData.json')
jsondata = jsonentry.read()
data = json.loads(jsondata)

In [41]:
data

{'version': {'json_export': '191',
  'page_segmentation': '5',
  'schema': '1.1.0',
  'structure': '1.1056.0',
  'table_structure': '5'},
 'extended_metadata': {'ID_instance': 'EA 0F 7D F4 10 B8 B2 11 0A 00 67 45 8B 6B C6 23 ',
  'ID_permanent': '45 42 20 36 36 20 32 38 20 46 34 20 31 30 20 42 38 20 42 32 20 31 31 20 30 41 20 30 30 20 36 37 20 34 35 20 38 42 20 36 42 20 43 36 20 32 33 20 ',
  'pdf_version': '1.6',
  'pdfa_compliance_level': '',
  'is_encrypted': False,
  'has_acroform': False,
  'is_digitally_signed': False,
  'pdfua_compliance_level': '',
  'page_count': 32,
  'has_embedded_files': False,
  'is_certified': False,
  'is_XFA': False,
  'language': 'en'},
 'elements': [{'Bounds': [51.32286071777344,
    778.878173828125,
    141.73941040039062,
    805.6040954589844],
   'ClipBounds': [51.45048522949219,
    778.878173828125,
    141.61224365234375,
    805.4764556884766],
   'Page': 0,
   'Path': '//Document/Reference/Figure',
   'attributes': {'BBox': [51.4778999999998

In [44]:
for element in data["elements"]:
    if(element["Path"].start("/H1")):
        print(element["Text"])


AttributeError: 'str' object has no attribute 'contain'